<a href="https://colab.research.google.com/github/Hansulich/park/blob/main/social_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model

url = "https://raw.githubusercontent.com/Hansulich/park/main/Social_Network_Ads.csv"
data = pd.read_csv(url)

# 데이터 전처리(preprocessing), Feature 추출
# "Age"(x1)와 "EstimatedSalary"(x2)를 특징 벡터(X)로 사용
X = data.iloc[:, [2, 3]].values  # Age와 EstimatedSalary 컬럼
y = data.iloc[:, 4].values       # Purchased 컬럼

# 학습 데이터를 데이터의 80%(320개), 테스트 데이터를 20%(80개)로 분할 사용
np.random.seed(42)
indices = np.random.permutation(len(X))
train_size = int(len(X) * 0.8)
train_indices = indices[:train_size]
test_indices = indices[train_size:]
X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

# 특징 스케일링을 통한 수렴성 향상(수동 구현)
def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std, mean, std

X_train, mean_train, std_train = standardize(X_train)
# 훈련 데이터의 평균과 표준편차로 테스트 데이터 스케일링
X_test = (X_test - mean_train) / std_train

# 4. 학습데이터에 대해 로지스틱회귀 모델 구성
# 예측 함수 : Sigmoid 함수 정의
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 경사하강법을 이용한 로지스틱 회귀 구현
def gradient_descent(X, y, learning_rate=0.01, iterations=10000):
    m, n = X.shape
    # 편향항 추가
    X_with_bias = np.c_[np.ones((m, 1)), X]
    # 가중치 초기화
    weights = np.zeros(n + 1)

    # 경사하강법
    for i in range(iterations):
        # 예측값 계산
        z = np.dot(X_with_bias, weights)
        predictions = sigmoid(z)

        # 그래디언트 계산
        gradient = np.dot(X_with_bias.T, (predictions - y)) / m

        # 가중치 업데이트
        weights = weights - learning_rate * gradient

        # 교차 엔트로피 오차 계산 (모니터링용, 선택사항)
        if i % 1000 == 0:
            z = np.dot(X_with_bias, weights)
            predictions = sigmoid(z)
            loss = -np.mean(y * np.log(predictions + 1e-15) + (1 - y) * np.log(1 - predictions + 1e-15))
            print(f"반복 횟수 {i}, 손실: {loss}")

    return weights

# 5. 경사하강법을 이용해서 모델 학습하고, 파라미터 값 출력
weights = gradient_descent(X_train, y_train)
w0, w1, w2 = weights
print(f"경사하강법 파라미터: w0 = {w0}, w1 = {w1}, w2 = {w2}")

# 6. 모델의 시각화
def plot_decision_boundary(X, y, weights, title):
    # 여백을 포함한 최소/최대값 설정
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # 격자 생성
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.01),
                          np.arange(x2_min, x2_max, 0.01))

    # 격자 점에 대한 예측
    X_grid = np.c_[xx1.ravel(), xx2.ravel()]
    X_grid_with_bias = np.c_[np.ones((X_grid.shape[0], 1)), X_grid]
    probs = sigmoid(np.dot(X_grid_with_bias, weights))
    probs = probs.reshape(xx1.shape)

    # 결정 경계선 시각화
    plt.figure(figsize=(10, 6))
    plt.contourf(xx1, xx2, probs, alpha=0.3, cmap=plt.cm.RdBu)
    plt.contour(xx1, xx2, probs, [0.5], linewidths=1, colors='black')

    # 데이터 포인트 시각화
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='red', label='구매 안함')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='green', label='구매함')

    plt.title(title)
    plt.xlabel('나이 (표준화)')
    plt.ylabel('예상 연봉 (표준화)')
    plt.legend()
    plt.grid(True)
    plt.show()

# 훈련 데이터에 대한 결정 경계선 플롯
plot_decision_boundary(X_train, y_train, weights, "로지스틱 회귀 (훈련 데이터) - 경사하강법")

# scikit-learn Library를 이용, 모델 학습하고, 파라미터 값 출력
sk_model = linear_model.LogisticRegression(random_state=42)
sk_model.fit(X_train, y_train)

# 파라미터 추출 (편향 = w0, 계수 = [w1, w2])
sk_w0 = sk_model.intercept_[0]
sk_w1, sk_w2 = sk_model.coef_[0]
print(f"Scikit-learn 파라미터: w0 = {sk_w0}, w1 = {sk_w1}, w2 = {sk_w2}")

# scikit-learn을 이용한 모델 시각화
# scikit-learn 모델을 위한 시각화 함수
def plot_decision_boundary_sklearn(X, y, model, title):
    # 여백을 포함한 최소/최대값 설정
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # 격자 생성
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.01),
                          np.arange(x2_min, x2_max, 0.01))

    # 격자 점에 대한 예측
    X_grid = np.c_[xx1.ravel(), xx2.ravel()]
    probs = model.predict_proba(X_grid)[:, 1]
    probs = probs.reshape(xx1.shape)

    # 결정 경계선 시각화
    plt.figure(figsize=(10, 6))
    plt.contourf(xx1, xx2, probs, alpha=0.3, cmap=plt.cm.RdBu)
    plt.contour(xx1, xx2, probs, [0.5], linewidths=1, colors='black')

    # 데이터 포인트 시각화
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='red', label='구매 안함')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='green', label='구매함')

    plt.title(title)
    plt.xlabel('나이 (표준화)')
    plt.ylabel('예상 연봉 (표준화)')
    plt.legend()
    plt.grid(True)
    plt.show()

# 훈련 데이터에 대한 시각화
plot_decision_boundary_sklearn(X_train, y_train, sk_model, '로지스틱 회귀 (훈련 데이터) - Scikit-learn')

# 테스트 데이터에 대한 시각화
plot_decision_boundary_sklearn(X_test, y_test, sk_model, '로지스틱 회귀 (테스트 데이터) - Scikit-learn')

# 추가 부분: 테스트 데이터에 대한 모델 평가 / 도움 받을것
y_pred = sk_model.predict(X_test)
accuracy = np.mean(y_test == y_pred)
print(f"테스트 데이터에 대한 모델 정확도: {accuracy * 100:.2f}%")

# 경사하강법 모델을 이용한 예측
X_test_with_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]
gd_predictions = sigmoid(np.dot(X_test_with_bias, weights))
gd_y_pred = (gd_predictions >= 0.5).astype(int)
gd_accuracy = np.mean(y_test == gd_y_pred)
print(f"경사하강법 모델의 테스트 데이터에 대한 정확도: {gd_accuracy * 100:.2f}%")

FileNotFoundError: [Errno 2] No such file or directory: 'Social_Network_Ads.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')